# QUIZ 17/08/23

In [1]:
import numpy as np

In [2]:
terminos = ['carro', 'compra', 'industrial', 'libro', 'maquina', 'mecanica', "nuevo", "usado", "venta"]
dfts = [1,1,2,2,3,1,2,3,3]
postings = [[1],[3],[3,5],[2,5],[3,4,5],[3],[2,5],[1,2,4],[1,2,4]]
doc1 = [1,0,0,0,0,0,0,1,1]
doc2 = [0,0,0,2,0,0,1,1,2]
doc3 = [0,1,1,0,2,1,0,0,0]
doc4 = [0,0,0,0,1,0,0,1,1]
doc5 = [0,0,1,1,0,1,1,0,0]
V_ = 9 # 9 palabras en vocabulario

In [3]:
def tf_idf(t, dt): 
    N = 5 #hay 5 documentos
    return np.log10(1+t)*np.log10(N/dt)

In [4]:
doc1_tfidf = [tf_idf(doc1[i], dfts[i]) for i in range(V_)]
doc2_tfidf = [tf_idf(doc2[i], dfts[i]) for i in range(V_)]
doc3_tfidf = [tf_idf(doc3[i], dfts[i]) for i in range(V_)]
doc4_tfidf = [tf_idf(doc4[i], dfts[i]) for i in range(V_)]
doc5_tfidf = [tf_idf(doc5[i], dfts[i]) for i in range(V_)]

In [5]:
# Concatenate the individual document TF-IDF lists into a single matrix
term_matrix = np.vstack((doc1_tfidf, doc2_tfidf, doc3_tfidf, doc4_tfidf, doc5_tfidf))

# Transpose the matrix to get a term-document matrix
term_document_matrix = term_matrix.T

In [6]:
term_document_matrix

array([[0.21041094, 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.21041094, 0.        , 0.        ],
       [0.        , 0.        , 0.11979188, 0.        , 0.11979188],
       [0.        , 0.18986564, 0.        , 0.        , 0.11979188],
       [0.        , 0.        , 0.10584875, 0.06678313, 0.        ],
       [0.        , 0.        , 0.21041094, 0.        , 0.21041094],
       [0.        , 0.11979188, 0.        , 0.        , 0.11979188],
       [0.06678313, 0.06678313, 0.        , 0.06678313, 0.        ],
       [0.06678313, 0.10584875, 0.        , 0.06678313, 0.        ]])

In [7]:
def compute_score(q, doc):
    score = 0
    for idx in range(len(doc)):
        if q[idx]!=0 and doc[idx]!=0:
            score += doc[idx]
    return score

In [8]:
#frase: compra de máquinas de helados
q    = np.array([0,0.21,0,0,0.066,0,0,0,0])

In [9]:
scores = []

scores.append(compute_score(q, doc1_tfidf))
scores.append(compute_score(q, doc2_tfidf))
scores.append(compute_score(q, doc3_tfidf))
scores.append(compute_score(q, doc4_tfidf))
scores.append(compute_score(q, doc5_tfidf))

In [10]:
# Use argsort to get the indices that would sort the array
sorted_indices = np.argsort(scores)

# Reverse the indices to get the descending order
ranked_indices = sorted_indices[::-1]

# Print the ranked indices and scores
for rank, index in enumerate(ranked_indices, start=1):
    print(f"Rank {rank}: Index {index}, Score {scores[index]}")

Rank 1: Index 2, Score 0.3162596911494689
Rank 2: Index 3, Score 0.06678312813507141
Rank 3: Index 4, Score 0
Rank 4: Index 1, Score 0
Rank 5: Index 0, Score 0


In [11]:
term_document_matrix

array([[0.21041094, 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.21041094, 0.        , 0.        ],
       [0.        , 0.        , 0.11979188, 0.        , 0.11979188],
       [0.        , 0.18986564, 0.        , 0.        , 0.11979188],
       [0.        , 0.        , 0.10584875, 0.06678313, 0.        ],
       [0.        , 0.        , 0.21041094, 0.        , 0.21041094],
       [0.        , 0.11979188, 0.        , 0.        , 0.11979188],
       [0.06678313, 0.06678313, 0.        , 0.06678313, 0.        ],
       [0.06678313, 0.10584875, 0.        , 0.06678313, 0.        ]])

In [12]:
transposed_tcm = term_document_matrix.T
M, N = transposed_tcm.shape
document_vectors = {}
# Iterate through each document
for doc_index in range(M):
    document_vector = transposed_tcm[doc_index, :]
    document_vectors[f"doc{doc_index+1}"] = document_vector

# Print the vectors
for doc_name, vector in document_vectors.items():
    print(f"{doc_name} vector:")
    print(vector)

doc1 vector:
[0.21041094 0.         0.         0.         0.         0.
 0.         0.06678313 0.06678313]
doc2 vector:
[0.         0.         0.         0.18986564 0.         0.
 0.11979188 0.06678313 0.10584875]
doc3 vector:
[0.         0.21041094 0.11979188 0.         0.10584875 0.21041094
 0.         0.         0.        ]
doc4 vector:
[0.         0.         0.         0.         0.06678313 0.
 0.         0.06678313 0.06678313]
doc5 vector:
[0.         0.         0.11979188 0.11979188 0.         0.21041094
 0.11979188 0.         0.        ]


In [13]:
def cosine_similarity(q,d):
    return np.dot(q,d) / (np.linalg.norm(q)*np.linalg.norm(d))

In [14]:
# Compute similarities and rank the documents
similarities = {}
for doc_name, doc_vector in document_vectors.items():
    similarity = cosine_similarity(q, doc_vector)
    similarities[doc_name] = similarity

# Rank the documents based on similarity (higher is better)
ranked_documents = sorted(similarities.items(), key=lambda x: x[1], reverse=True)

# Print the ranked documents
for rank, (doc_name, similarity) in enumerate(ranked_documents, start=1):
    print(f"Rank {rank}: Document {doc_name}, Similarity {similarity:.4f}")

Rank 1: Document doc3, Similarity 0.6882
Rank 2: Document doc4, Similarity 0.1731
Rank 3: Document doc1, Similarity 0.0000
Rank 4: Document doc2, Similarity 0.0000
Rank 5: Document doc5, Similarity 0.0000
